In [1]:
import numpy as np
import random
import itertools


In [2]:
### I threat the total length of time as the length of the observered list

class HMM:
    def __init__(self,no_of_state,no_of_value,states,values,observered=None):
        #randomize all matrix, each row should sum to 1
        self.emssion_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_value) for i in range(no_of_state)])]) #b[i][O]  --> probability to emit values[O] from states[i]
        
        self.initial_matrix= np.random.rand(no_of_state) #π[i] --> probability to start at states[i]
        self.initial_matrix/= self.initial_matrix.sum()
        
        self.transition_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_state)])])  #a[i][j] --> probability from states[i] transit to states[j]
        
        self.states=states
        self.values=values
        self.observered=observered
        
    def debug(self):
        print('initial_matrix\n',self.initial_matrix)
        print('transition_matrix\n',self.transition_matrix)
        print('emssion_matrix\n',self.emssion_matrix)
        
    def forward(self,t,j,obs=None,mode='eval'):
        
        if obs is None:
            obs=self.observered
        if t==0:
            return self.initial_matrix[j]*self.emssion_matrix[j][obs[t]]
        else:
            if mode=='eval':
                return sum([self.forward(t-1,i,obs)*self.transition_matrix[i][j] for i in range(len(self.states))]) * self.emssion_matrix[j][obs[t]]
            elif mode=='decode':
                return max([self.forward(t-1,i,obs)*self.transition_matrix[i][j] for i in range(len(self.states))]) * self.emssion_matrix[j][obs[t]]
            
    def backward(self,t,i,obs=None,mode='eval'):
        if obs is None:
            obs=self.observered
        if t==len(obs)-1:
            return 1
        else:
            if mode=='eval':
                return sum([self.transition_matrix[i][j]*self.emssion_matrix[j][obs[t+1]]*self.backward(t+1,j,obs) for j in range(len(self.states))])
            elif mode=='decode':
                return max([self.transition_matrix[i][j]*self.emssion_matrix[j][obs[t+1]]*self.backward(t+1,j,obs) for j in range(len(self.states))])
            
    def train(self,obs=None,epochs=2):
        #O:observered values
        #λ:model parameters
         
        if obs is None:
            obs=self.observered
        def probit_at_i(t,i,ob):#Gamma γt(i) = P(qt = i|O,λ)      
            numerator=self.forward(t,i,ob)*self.backward(t,i,ob)#sum probability of all path passing through state[i] at time t
            denominator=sum([self.forward(t,j,ob)*self.backward(t,j,ob) for j in range(len(self.states))]) #prob of passing through  ALL_state at time t
            return numerator/denominator
        def probit_transit_i_j(t,i,j,ob):#epsilon ξt(i, j) = P(qt = i,qt+1 = j|O,λ)
            numerator=self.forward(t,i,ob)*self.transition_matrix[i][j]*self.emssion_matrix[j][ob[t+1]]*self.backward(t+1,j,ob)#sum probability of all path transit from state[i] to state[j] at time t
            denominator=sum([sum([self.forward(t,m,ob)*self.transition_matrix[m][n]*self.emssion_matrix[n][ob[t+1]]*self.backward(t+1,n,ob) for n in range(len(self.states))]) for m in range(len(self.states))]) #prob of ALL transition combination at time t
            return numerator/denominator
        
        for epoch in range(epochs):
            for ob in obs:
                for i in range(len(self.states)):
                    self.initial_matrix[i]=probit_at_i(0,i,ob)
                for i, j in itertools.product(range(len(self.states)),range(len(self.states))):
                    self.transition_matrix[i][j]=sum([probit_transit_i_j(t,i,j,ob) for t in range(len(ob)-1)])/sum([probit_at_i(t,i,ob) for t in range(len(ob)-1)])
                for j, k in itertools.product(range(len(self.states)),range(len(self.values))):   
                    total=0
                    for t in range(len(ob)):
                        if ob[t]==k:
                            total+=probit_at_i(t,j,ob)
                    self.emssion_matrix[j][k]=total/sum([probit_at_i(t,j,ob) for t in range(len(ob))])

In [3]:
test=HMM(2,3,["Rainy", "Sunny"],["walk", "shop", "clean"])

In [4]:
test.train([[0,2,0, 1, 2, 0],[1,0,1,1,1,2],[2, 1, 2, 1, 2, 0]],1)

In [5]:
test.debug()

initial_matrix
 [0.14177705 0.95457669]
transition_matrix
 [[0.10010287 0.95824144]
 [0.10699402 0.90332251]]
emssion_matrix
 [[0.25965211 0.59739574 0.16517015]
 [0.16352431 0.33099158 0.52297268]]


## possible modification:
observe more than 1 value at each timestamp